## Getting Started with the Maxar Geospatial Platform Monitoring API

This sample workflow will guide a user through using the Maxar Geospatial Platform monitoring API with real world examples. The process will go through:
* Validating a monitor
* Creating a monitor
* Listing monitors and monitor events
* Toggling a monitor between enable and disable

#### Authentication
For a more detailed view of how to authorize with MGP, please see the *Auth and Token Service* notebook in this collection

In [ ]:
import requests
import json

username = "your.email@address.com"
password= "yourpassword"

url = "https://account.maxar.com/auth/realms/mds/protocol/openid-connect/token"

payload = 'client_id=mgp&username={}&password={}&grant_type=password'.format(username, password)
headers = {'Content-Type': 'application/x-www-form-urlencoded'}

response = requests.request("POST", url, headers=headers, data=payload)

access_token = {'Content-Type': 'application/json', 'Authorization': 'Bearer {}'.format(response.json()['access_token'])}

#### Creating a new monitor

Before a new monitor can be created, some variables will need to be established to make the submission process easier. Note that monitors must be set to a time at least 15 minutes into the future.

In [ ]:
import datetime

now_start = datetime.datetime.utcnow() + datetime.timedelta(minutes=20)
now_time = now_start.isoformat()
end_start = datetime.datetime.utcnow() + datetime.timedelta(days=1)
end_time = end_start.isoformat()
now = now_time.replace(now_time[-7:], "+00:00")
end = end_time.replace(end_time[-7:], "+00:00")
print(now)
print(end)

The monitoring API utilizes GeoJSON formatts for an Area Of Interest (AOI).

In [ ]:
geojson = {
    "type": "Polygon",
    "coordinates": [
        [
            [-106.8, 35.1],
            [-106.4, 35.1],
            [-106.4, 35.4],
            [-106.8, 35.4],
            [-106.8, 35.1]
        ]
    ]
}

The match criteria will be set to find features taken by either WorldView 2 or WorldView 3 with a cloud cover less than 75%.

In [ ]:
criteria = {
    "platform": {
        "in": ["worldview-03", "worldview-02"]
    },
    "eo:cloud_cover": {
        "lt": 75
    },
    "aoi:coverage_sqkm": {
        "gte": 1
    }
}

When a monitor finds imagery over the desired AOI, a notification will be sent to the user. This example will set the notification schema to email a user when imagery matching the desired criteria is found.

In [ ]:
notifications = [
    {
        "type": "email",
        "target": "your.email@address.com"
    }
]

With the variables defined, the monitor request is almost ready be submitted. Imagery will be searched for in the discovery catalog. Firstly, a user should verify that everything is constructed correctly with a verification call.

In [ ]:
monitor_verify_url = "https://api.maxar.com/monitoring/v1/monitors?validation_only=true"

monitor_verify_payload = json.dumps({"source": "discovery/catalog", "start_datetime": now, "end_datetime": end, "description": 
                                    "Your Monitor Description", "aoi_geojson": geojson, "match_criteria": criteria, 
                                    "monitor_notifications": notifications})

monitor_verify_response = requests.request("POST", monitor_verify_url, headers=access_token, data=monitor_verify_payload)

print(monitor_verify_response.json())

Now that the monitor has passed the validation, the monitor request can be submitted.

In [ ]:
monitor_url = "https://api.maxar.com/monitoring/v1/monitors"

monitor_payload = json.dumps({"source": "discovery/catalog", "start_datetime": now, "end_datetime": end, "description": 
                              "Your Monitor Description", "aoi_geojson": geojson, "match_criteria": criteria, 
                              "monitor_notifications": notifications})

monitor_response = requests.request("POST", monitor_url, headers=access_token, data=monitor_payload)

print(monitor_response.json())

#### Monitor details

With the monitor submission created, a user can now look at the details for the monitor by taking the monitor id and passing it into the following request.

In [ ]:
new_monitor = monitor_response.json()['data']['id']

get_monitor_url = "https://api.maxar.com/monitoring/v1/monitors/{}".format(new_monitor)

get_monitor_response = requests.request("GET", get_monitor_url, headers=access_token)

print(get_monitor_response.json())

After a monitor has been submitted, a user can look at the monitor events as they have taken place with the `List Monitor Events` request.

In [ ]:
monitor_events_url = "https://api.maxar.com/monitoring/v1/monitors/{}/events".format(new_monitor)

monitor_events_response = requests.request("GET", monitor_events_url, headers=access_token)

print(monitor_events_response.json())

#### Toggle monitors

There may be times that a user wishes to disable an enabled monitor before the initial end time, or enable a disabled monitor that was disabled prematurely. This can be done with the following request. *Note: Disabled monitors can only be enabled if the end datetime hasn't been reached yet. Enabled monitors can only be disabled if the start datetime has already happened*

In [ ]:
disable_monitor_url = "https://api.maxar.com/monitoring/v1/monitors/{}/disable".format(new_monitor)

disable_monitor_response = requests.request("POST", disable_monitor_url, headers=access_token)

print(disable_monitor_response.json())

In [ ]:
enable_monitor_url = "https://api.maxar.com/monitoring/v1/monitors/{}/enable".format(new_monitor)

enable_monitor_response = requests.request("POST", enable_monitor_url, headers=access_token)

print(enable_monitor_response.json())